# Eniac

## 1.Importing Data

Turning them into a dictionary of dataframes.

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

orders_url = "https://raw.githubusercontent.com/MerleSt/Eniac/main/Data-Eniac/Data_Cleaned/orders_clean.parquet"
orderlines_url = "https://raw.githubusercontent.com/MerleSt/Eniac/main/Data-Eniac/Data_Cleaned/orderlines_clean.parquet"
products_url = "https://raw.githubusercontent.com/MerleSt/Eniac/main/Data-Eniac/Data_Cleaned/products_clean.parquet"
brands_url = "https://raw.githubusercontent.com/MerleSt/Eniac/main/Data-Eniac/Data_Cleaned/brands_clean.parquet"

# Loading dataframes directly from GitHub
orders_df = pd.read_parquet(orders_url)
orderlines_df = pd.read_parquet(orderlines_url)
products_df = pd.read_parquet(products_url)
brands_df = pd.read_parquet(brands_url)

orders = orders_df.copy()
orderlines = orderlines_df.copy()
products = products_df.copy()
brands = brands_df.copy()

Set Display Formats of Pandas

In [57]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_rows', 100)

## 2. Data Integration

### 2.1 Mergers and logical combinations

In [58]:
# Rename column to match with orders (better readability)
orderlines.rename(columns={'id_order': 'order_id'}, inplace=True)
orderlines['short'] = orderlines['sku'].str[:3]

In [59]:
oo = orders.merge(orderlines, on='order_id', how='right')
boo = oo.merge(brands, on='short', how='left')

Delete oders where there is one or more products, which are not identified. 
1.make a list of all known products (products table). Go into the orders, orderlines, brands table and exclude orders whose id is not in that list. Lastly merge the products table onto the boo table.

In [60]:
known_products = products['sku']
orders_with_unknown_products = boo.loc[~boo.sku.isin(known_products), "order_id"]
boo = boo.loc[~boo["order_id"].isin(orders_with_unknown_products), :]
df = boo.merge(products, on ='sku', how='left')

### 2.2 Create new columns

#### 2.2.1 Boolean Promotion and Promo_price

Create a boolean condition which is true when a promotion was applied on the price.

In [61]:
df['price_promotion'] = df['unit_price'] < df['price']
df['promo_price'] = df.apply(lambda row: row['unit_price'] if row['price_promotion'] else np.nan, axis=1)
df[['promo_price','price_promotion', 'price']].head()

,promo_price,price_promotion,price
0,18.99,True,34.99
1,399.00,True,429.00
2,474.05,True,699.00
3,68.39,True,79.00
4,23.74,True,29.90


#### 2.2.2 order_price

Use this price to compare to the total paid, to infer knowledge about shipping costs etc.

In [62]:
df['unit_price_total'] = df['product_quantity'] * df['unit_price']
order_total_price = df.groupby('order_id')['unit_price_total'].sum().reset_index()
order_total_price.rename(columns={'unit_price_total': 'order_price'}, inplace=True)
df = df.merge(order_total_price, on='order_id')

#### 2.2.3 Orders with at least one promotional item

In [63]:
df['order_inc_promo'] = df.groupby('order_id')['price_promotion'].transform('any')

### 2.3 Outliers

### 2.4 Second Dataframe, with completed orders

Do this as a laast step so all columns and the cleaning/ merging are the same in both dataframes

In [ ]:
df_com = df.loc[df['state']=='Completed'].reset_index(drop=True)
df_com.head()

## 3. Analysis

###3. Analyzing Order Patterns in Relation to Discounts

Objective:
To understand if the introduction or availability of discounts affects order patterns – both in terms of quantity and frequency.

### 1. Understand basic metrics

In [45]:
total_orders = df_com['order_id'].nunique()
avg_order_value = df_com.groupby('order_id')['total_paid'].first().mean()

avg_items_per_order = df_com.groupby('order_id').size().mean()
print(f'The total orders are: {total_orders}, the average order values is: {avg_order_value}, and the average items per order are: {avg_items_per_order}')


The total orders are: 44852, the average order values is: 333.69310733077674, and the average items per order are: 1.3166636939266922


Let me check if all total_paid values are the same within each order

In [46]:
unique_paid_counts = df_com.groupby('order_id')['total_paid'].nunique()
inconsistent_orders = unique_paid_counts[unique_paid_counts > 1]
if not inconsistent_orders.empty:
    print("There are orders with inconsistent 'total_paid' values:")
    print(inconsistent_orders)
else:
    print("All orders have consistent 'total_paid' values.")

All orders have consistent 'total_paid' values.


### 3. Compare Metrics

In [68]:
avg_order_value_discounted =(
    df[df['order_inc_promo'] == True]
    .groupby('order_id')['total_paid']
    .sum()
    .mean()
    .round(2)
)
avg_order_value_non_discounted =(
    df[df['order_inc_promo'] == False]
    .groupby('order_id')['total_paid']
    .sum()
    .mean()
    .round(2)
)

In [69]:
avg_items_discounted =(
    df[df['order_inc_promo'] == True]
    .groupby('order_id')['product_quantity']
    .sum()
    .mean()
    .round(2)
)
avg_items_non_discounted =(
    df[df['order_inc_promo'] == False]
    .groupby('order_id')['product_quantity']
    .sum()
    .mean()
    .round(2)
)

In [70]:
print(f'The average order value of discounted orders is: {avg_order_value_discounted}, compared to the average value of non discounted orders: {avg_order_value_non_discounted}\
. Moreover, the average number of items in an order with prmotion is: {avg_items_discounted}, while it is : {avg_items_non_discounted} for non discounted orders.')

The average order value of discounted orders is: 1667.65, compared to the average value of non discounted orders: 391.74. Moreover, the average number of items in an order with prmotion is: 1.61, while it is : 1.23 for non discounted orders.


### 4. Temporal Analysis

For now only use completed orders, since otherwise we cannot infer about the promotion.

In [ ]:
df.dtypes

In [ ]:
df_com['created_month'] = df_com['created_date'].dt.month
monthly_discounted_orders = df_com[df_com['promotion']].groupby('created_month').size()
monthly_discounted_orders

### 5. Visual Analysis

In [ ]:
import matplotlib.pyplot as plt

# Average order values comparison
plt.bar(['Discounted', 'Non-Discounted'], [avg_order_value_discounted, avg_order_value_non_discounted])
plt.title('Average Order Values')
plt.ylabel('Value')
plt.show()

# Monthly trends of discounted orders
monthly_discounted_orders.plot(kind='bar')
plt.title('Monthly Trend of Discounted Orders')
plt.ylabel('Number of Orders')
plt.xlabel('Month')
plt.show()


Possible Conclusions:

- If orders with discounts have a significantly higher average value or quantity, it suggests discounts might be driving larger orders.
- If there's a temporal trend showing increased discount orders during specific times, it might suggest seasonal trends or the effects of periodic discount campaigns.

### Which Items are bought together

In [83]:
df['grouped'] = df.groupby('order_id')['name'].transform(lambda x: ','.join(x))

In [89]:
from itertools import combinations
from collections import Counter

count = Counter()

for row in df['grouped']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list, 2)))
    
# Print only the first few, since dataframe is too large.
for item, freq in count.most_common(10):
    print(item, freq)

('IMac AppleCare Protection Plan', 'Office 365 Personal 1 license 1 M ± or') 15272
('Apple Magic Trackpad 2', 'Office 365 Personal 1 license 1 M ± or') 14705
('IMac AppleCare Protection Plan', 'Apple Magic Trackpad 2') 14172
('My MW Case MacBook Pro 13 "(Late 2016) Black', 'AppleCare Protection Plan MacBook / Air / Pro 13 "') 6524
('Office 365 Personal 1 license 1 M ± or', 'Apple Magic Trackpad 2') 6349
('Satechi Hub Multiport USB-C 4K HDMI Space Gray', 'My MW Case MacBook Pro 13 "(Late 2016) Black') 4746
('Satechi Hub Multiport USB-C 4K HDMI Space Gray', 'AppleCare Protection Plan MacBook / Air / Pro 13 "') 4728
('Satechi Multiport USB Hub HDMI 4K-C Plata', 'AppleCare Protection Plan MacBook / Air / Pro 13 "') 4720
('Satechi Multiport USB Hub HDMI 4K-C Plata', 'My MW Case MacBook Pro 13 "(Late 2016) Black') 4705
('Apple Magic Trackpad 2', 'IMac AppleCare Protection Plan') 3538


## Step 3: Analyzing Customer Behavior and Retention with Respect to Discounts:

### 1. Retention Rate with Respect to Discounts:

To understand if customers acquired during promotional periods are more likely to come back.

In [ ]:
df_com.groupby('order_id')['product_quantity'].sum()

In [ ]:
duplicate_order_ids = df_com[df_com['order_id'].duplicated(keep=False)]

if not duplicate_order_ids.empty:
    print(f"There are {len(duplicate_order_ids)} duplicated order_ids in the dataset.")
else:
    print("All order_ids are unique.")


In [ ]:
df_com.info()

Calculate the percentage of promotion for every product in every order: (1-(price/unit_price))

In [ ]:
df_com['promo_perc'] = (1-(df_com['unit_price']/df_com['price']))

Average promotion perc per week/ month

In [ ]:
df_com[['year', 'week', 'day']]= df_com['created_date'].dt.isocalendar()
# Extract month and quarter
df_com['month'] = df_com['created_date'].dt.month
df_com['quarter'] = df_com['created_date'].dt.quarter

In [ ]:
grouped = (
    df_com
    .loc[df_com['state'] == 'Completed']
    .groupby(['week', 'month', 'quarter', 'year'])
    ['promo_perc']
    .mean()
    .reset_index()
)

grouped

In [ ]:
import matplotlib.pyplot as plt

# Creating 'week-year' for plotting
grouped['week-year'] = grouped['week'].astype(str) + '-' + grouped['year'].astype(str)

# Group by 'week-year' and calculate the mean of 'promo_perc'
weekly_promo = grouped.groupby('week-year')['promo_perc'].mean()

# Plot
plt.figure(figsize=(15, 7))
weekly_promo.plot(color='steelblue')
plt.title('Average Promotional Percentage Over Weeks', fontsize=18)
plt.xlabel('Week-Year', fontsize=15)
plt.ylabel('Average Promotional Percentage', fontsize=15)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
 grouped =(
     df_com
     .loc[df_com['state']=='Completed']
     .groupby([df_com['created_date'].dt.isocalendar().week, df_com['created_date'].dt.month, df_com['created_date'].dt.quarter, df_com['created_date'].dt.isocalendar().year])
     ['promo_perc']
     .mean()
 )
#grouped.columns = ['week', 'month', 'quarter', 'promo_perc']
grouped.columns = ['week', 'month', 'quarter', 'promo_perc']
print(grouped)

Make the Line Plot

look for weeks/months/days where promotion in percentage/ price high. Which one to use?